# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [72]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json
import requests

# Import API key
from api_keys import g_key
from api_keys import weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
cities_df = pd.read_csv('./citiesA.csv')
cities_df.dropna()
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Mataura,44,NZ,1583963094,85,-46.19,168.86,48.38,3.04
1,1,Esperance,43,AU,1583962860,82,-33.87,121.90,60.80,8.05
2,2,Port Alfred,100,ZA,1583963094,91,-33.59,26.89,68.00,3.00
3,3,Omsukchan,100,RU,1583963094,98,62.53,155.80,-5.57,11.27
4,4,Geraldton,75,AU,1583963094,83,-28.77,114.60,69.80,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

0      85.0
1      82.0
2      91.0
3      98.0
4      83.0
      ...  
86     90.0
87     44.0
88     96.0
89     92.0
90    100.0
Name: Humidity, Length: 91, dtype: float64

In [69]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
#coordinates = cities_df[["Lat", "Lng"]]

In [85]:
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout,center=(50.0, 50.0), zoom_level=1.85)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius=1)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(coordinates, weights=humidity))

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
hotel_df =   cities_df[(cities_df['Max Temp'] >=50) 
          & (cities_df['Max Temp'] < 80) 
          & (cities_df['Wind Speed'] < 10)
          & (cities_df['Cloudiness'] == 0)].reset_index()

narrowed_city_df =   cities_df[(cities_df['Max Temp'] >=50) 
          & (cities_df['Max Temp'] < 80) 
          & (cities_df['Wind Speed'] < 10)
          & (cities_df['Cloudiness'] == 0)]


hotel_df.insert(loc=1, column='Hotel Name', value=['' for i in range(narrowed_city_df.shape[0])])
hotel_df

,index,Hotel Name,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,55,,55,Castro,0,BR,1583963103,63,-24.79,-50.01,72.82,2.82
1,77,,77,Kyaikto,0,MM,1583963108,81,17.30,97.02,63.16,4.05
2,90,,90,Carnarvon,0,AU,1583963110,100,-24.87,113.63,69.80,6.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    location = f"{row['Lat']}, {row['Lng']}"
    params = {
        "location": location,  # city coords,
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
    }
    #print(params)
    
    response = requests.get(base_url, params=params).json()

    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    #print(response.url)
    #print(json.dumps(response, indent=4, sort_keys=True))
   
    # extract results
    results = response['results']
   
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = {results[0]['name']}      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
#hotel_df['Hotel Name']

Closest Hotel is QUARTOS CASTRO CENTRO.
------------
Closest Hotel is သက်တန့်အိမ်.
------------
Closest Hotel is Tilly Gia home sweet home.
------------


In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations
    

,Lat,Lng
0,-24.79,-50.01
1,17.30,97.02
2,-24.87,113.63


In [64]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_df['Hotel Name'])

# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### End of notebook